<a href="https://colab.research.google.com/github/brunotiglla/TP-1/blob/main/Pre-Models/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingsound

In [ ]:
from huggingsound import SpeechRecognitionModel

model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-spanish")
audio_paths = ["5.ogg"]

transcriptions = model.transcribe(audio_paths)

INFO:huggingsound.speech_recognition.model:Loading model...


100%|██████████| 1/1 [00:02<00:00,  3.00s/it]


In [ ]:
import torch
import librosa
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

LANG_ID = "es"
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-spanish"
SAMPLES = 10

test_dataset = load_dataset("common_voice", LANG_ID, split=f"test[:{SAMPLES}]")

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

# Preprocessing the datasets.
# We need to read the audio files as arrays
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = librosa.load(batch["path"], sr=16_000)
    batch["speech"] = speech_array
    batch["sentence"] = batch["sentence"].upper()
    return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)
predicted_sentences = processor.batch_decode(predicted_ids)

for i, predicted_sentence in enumerate(predicted_sentences):
    print("-" * 100)
    print("Reference:", test_dataset[i]["sentence"])
    print("Prediction:", predicted_sentence)


/root/.cache/huggingface/modules/datasets_modules/datasets/common_voice/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf/common_voice.py:634: FutureWarning: 
            This version of the Common Voice dataset is deprecated.
            You can download the latest one with
            >>> load_dataset("mozilla-foundation/common_voice_11_0", "en")
            
  warnings.warn(


Generating train split:   0%|          | 0/161813 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/15089 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/15089 [00:00<?, ? examples/s]

Generating other split:   0%|          | 0/144791 [00:00<?, ? examples/s]

Generating validated split:   0%|          | 0/236314 [00:00<?, ? examples/s]

Generating invalidated split:   0%|          | 0/40640 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

----------------------------------------------------------------------------------------------------
Reference: BIEN . ¿ Y QUÉ REGALO VAS A ABRIR PRIMERO ?
Prediction: bien y qué regalo vas a abrir primero
----------------------------------------------------------------------------------------------------
Reference: ESTAMOS CONSTRUYENDO UNA PLATAFORMA DE LANZAMIENTO .
Prediction: estamos construyendo una plataforma de lanzamiento
----------------------------------------------------------------------------------------------------
Reference: ¡ CINCUENTA PULSACIONES Y BAJANDO !
Prediction: cincuenta pulsaciones y bajando
----------------------------------------------------------------------------------------------------
Reference: LA SOMBRA NEGRUZCA DE UN INDIO ASOMÓ EN LA PUERTA :
Prediction: la sombra negruzca de un indio asomó en la puerta
----------------------------------------------------------------------------------------------------
Reference: OS PARECÉIS MUCHÍSIMO.
Prediction: o

In [ ]:
test_dataset

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'speech'],
    num_rows: 10
})

In [ ]:
model